In [141]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
from src import pyfunctions

In [132]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
pwd

'/Users/travis/Documents/Education/SMU/OneDrive - Southern Methodist University/Code/ds6371_group'

In [3]:
ls data

backward.tsv                stepwise.tsv
backward_params.csv         stepwise_params.csv
custom_columns              test_withdummy.csv
custom_transforms.json      train_withdummy.csv
forward.tsv                 train_withdummy_pandas.csv
forward_params.csv


In [4]:
backwards = pd.read_csv('data/backward_params.csv')

In [5]:
backwards.head()

,Parameter,Estimate
0,Intercept,3.638448
1,IntLotFrontage,-0.000231
2,MSSubClass,-0.000139
3,MSZoning C,-0.363794
4,MSZoning FV,0.089339


In [43]:
test = pd.read_csv('test.csv')
dummy_test = pd.read_csv('data/test_withdummy.csv')

In [10]:
test.sample(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
1204,2665,20,RL,61.0,7740,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2006,New,Partial
92,1553,70,RM,60.0,12900,Pave,Grvl,Reg,Lvl,AllPub,...,168,0,NaN,NaN,NaN,0,5,2010,WD,Normal
1063,2524,80,RL,59.0,8660,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,7,2007,WD,Normal
1153,2614,20,RL,62.0,10447,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,12,2007,WD,Normal
1159,2620,60,RL,60.0,8238,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,5,2006,WD,Normal


In [9]:
dummy_test.head()

,Unnamed: 0,Id,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType$ConLw,SaleType$ConLI,SaleType$CWD,SaleType$nan,SaleCondition$Normal,SaleCondition$Partial,SaleCondition$Abnorml,SaleCondition$Family,SaleCondition$Alloca,SaleCondition$AdjLand
0,0,1461,80.0,11622,5,6,1961,1961,0.0,468.0,...,False,False,False,False,True,False,False,False,False,False
1,1,1462,81.0,14267,6,6,1958,1958,108.0,923.0,...,False,False,False,False,True,False,False,False,False,False
2,2,1463,74.0,13830,5,5,1997,1998,0.0,791.0,...,False,False,False,False,True,False,False,False,False,False
3,3,1464,78.0,9978,6,6,1998,1998,20.0,602.0,...,False,False,False,False,True,False,False,False,False,False
4,4,1465,43.0,5005,8,5,1992,1992,0.0,263.0,...,False,False,False,False,True,False,False,False,False,False


In [35]:
float(pd.Series(backwards.index).apply(lambda x: True if x in dumcols else False).sum()) / \
float(backwards.index.size)

0.8373983739837398

In [159]:
backwards = pd.read_csv('data/backward_params.csv')
test = pd.read_csv('test.csv')
idx = ['#'.join(var.split()) for var in backwards['Parameter'].values]
backwards.index = idx

backthings = pyfunctions.bring_dummy_to_test(backwards, test)

In [161]:
forewards = pd.read_csv('data/forward_params.csv')
test = pd.read_csv('test.csv')
idx = ['#'.join(var.split()) for var in forewards['Parameter'].values]
forewards.index = idx

forethings = pyfunctions.bring_dummy_to_test(forewards, test)

In [162]:
forethings.columns == forewards.index

array([ True,  True,  True,  True,  True,  True,  True], dtype=bool)

In [163]:
stepwise = pd.read_csv('data/stepwise_params.csv')
test = pd.read_csv('test.csv')
idx = ['#'.join(var.split()) for var in stepwise['Parameter'].values]
stepwise.index = idx

stepthings = pyfunctions.bring_dummy_to_test(stepwise, test)

In [180]:
swise = stepwise['Estimate']

new_df = stepthings.copy()
for col in stepthings.columns:
    new_df[col] = new_df[col] * swise[col]
    
step_predictions = np.apply_along_axis(sum, 1, new_df.values)

In [196]:
def get_predictions(params, clean_test):
    pest = params['Estimate']
    new_df = clean_test.copy()
    for col in clean_test.columns:
        new_df[col] = new_df[col] * pest[col]
    return np.exp(new_df.T.sum())

In [197]:
back_predictions = get_predictions(backwards, backthings)

In [198]:
back_predictions.to_csv('data/backward_predictions.csv')

In [202]:
step_predictions = get_predictions(stepwise, stepthings)

In [204]:
step_predictions.to_csv('data/stepwise_predictions.csv')

In [205]:
foreward_predictions = get_predictions(forewards, forethings)

In [206]:
foreward_predictions.to_csv('data/foreward_predictions.csv')

In [207]:
ls data

backward.tsv                forward_params.csv
backward_params.csv         stepwise.tsv
backward_predictions.csv    stepwise_params.csv
custom_columns              stepwise_predictions.csv
custom_params.tsv           test_withdummy.csv
custom_transforms.json      train_withdummy.csv
foreward_predictions.csv    train_withdummy_pandas.csv
forward.tsv


In [209]:
# pyfunctions.convert_tsv('data/custom_params.tsv', 'data/custom_params.csv')

In [ ]:
custom = pd.read_csv('data/custom_params.csv')
# test = pd.read_csv('test.csv')
idx = ['#'.join(var.split()) for var in custom['Parameter'].values]
stepwise.index = idx

stepthings = pyfunctions.bring_dummy_to_test(stepwise, test)

In [ ]:
custom_predictions = get_predictions()